# Custom Corolectal Cancer Pre-trained Model

### This notebook contains the implementation for the first pre-trained CNN encoder outlined as the first task of the project proposal. The architecture selected is ResNet and the implementation used comes from the `torchvision` module.

### Below are 4 different attempts at pre-training the model to find the ideal loop, optimizer and loss function that will yield the best performance for the model. The following 4 optimizers were selected.
- #### **SGD**
- #### **SGD + Momentum**
- #### **RMSProp**
- #### **Adam**

### Each loop utilizes the Categorical cross Entropy (CCE) loss function due to the presence of 3 distinct classes in the dataset (MUS, NORM, STR)

#### Initializations

In [14]:
import os
import numpy as np
import torch
import torch.nn as nn
from torchvision.models import resnet50
from torch.optim import SGD, Adam, RMSprop
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from PIL import Image
import math

print(f"torch.cuda.is_available(): {torch.cuda.is_available()}")

torch.manual_seed(0)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# [Task 1]: Import and initialize ResNet50
resnet_corolectal_cancer_model = resnet50(weights=None)
'''
Note that by default the ResNet implementations from PyTorch's source code follow the ImaegNet fully connected layer specification of 2048x1000.
    Therefore, for our project, we need to change it to use 3 classes according to the first dataset (MUS, NORM, STR).
    See the source code for the base class here (https://github.com/pytorch/vision/blob/main/torchvision/models/resnet.py).
    Stack Overflow post here (https://stackoverflow.com/questions/68980724/the-number-of-classes-in-pytorch-pretrained-model).

    Additionally, we will need to remove the last layer to be able to pass the features to a different learning algorithm in task 2.
    Here is a Stack Overflow post about it (https://stackoverflow.com/questions/52548174/how-to-remove-the-last-fc-layer-from-a-resnet-model-in-pytorch).

'''
resnet_corolectal_cancer_model.fc = torch.nn.Linear(2048, 3)
resnet_corolectal_cancer_model.to(device=device)
resnet_corolectal_cancer_model.train()

# [Task 1]: Import and initialize the optimizer algorithms
'''
Hyperparameters for vanilla SGD recommended settings: learning rate = 0.001
                                                      L2 regularizer penalty = 1e-6
                                                      Batch Size = 100
'''
learning_rate = 0.001
l2_regularizer = 1e-6
batch_size = 32
num_epochs = 1 

sgd_no_momentum = SGD(resnet_corolectal_cancer_model.parameters(), lr=learning_rate, weight_decay=l2_regularizer, differentiable=True)

'''
Hyperparameters for SGD + momentum recommended settings: learning rate = 0.001
                                                         L2 regularizer penalty = 1e-6
                                                         Batch Size = 100
                                                         Momentum = 0.9
                                                         Nesterov = true
'''
sgd_momentum = 0.9
sgd_momentum_use_nesterov = True

sgd_momentum = SGD(resnet_corolectal_cancer_model.parameters(),
                   lr=learning_rate,
                   weight_decay=l2_regularizer,
                   momentum=sgd_momentum,
                   nesterov=sgd_momentum_use_nesterov,
                   differentiable=True)

'''
Hyperparameters for Adam recommended settings: learning rate = 0.001
                                               L2 regularizer penalty = 1e-6
                                               Batch Size = 100
                                               Beta 1 (Momentum) = 0.9
                                               Beta 2 (RMS decay rate) = 0.999
                                               Division stablizer = 1e-8
'''
adam_betas = (0.9, 0.999)
epsilon_division_stablizer = 1e-8

adam = Adam(resnet_corolectal_cancer_model.parameters(),
            lr=learning_rate,
            weight_decay=l2_regularizer,
            betas=adam_betas,
            eps=epsilon_division_stablizer,
            differentiable=True)

'''
Hyperparameters for RSMProp recommended settings: learning rate = 0.001
                                                  L2 regularizer penalty = 1e-6
                                                  Batch Size = 100
                                                  Decay rate = 0.999
                                                  Division stablizer = 1e-8
'''
rmsprop_decay_rate = adam_betas[1]

rmsprop = RMSprop(resnet_corolectal_cancer_model.parameters(),
                  lr=learning_rate,
                  weight_decay=l2_regularizer,
                  alpha=rmsprop_decay_rate,
                  eps=epsilon_division_stablizer,
                  differentiable=True)

# [Task 1]: Import and initialize the t-SNE visualization algorithm
tsne = TSNE(random_state=42)

torch.cuda.is_available(): True


#### Data Preprocessing

In [3]:
'''
Source(s):
    - Key Steps in Data Preprocessing: https://www.techtarget.com/searchdatamanagement/definition/data-preprocessing 
'''
# Optimizers
optimizer = RMSprop(resnet_corolectal_cancer_model.parameters(), lr=learning_rate, weight_decay=l2_regularizer)

# Preprocessing transformations
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # ResNet
    transforms.RandomHorizontalFlip(),  # Data augmentation
    transforms.ToTensor(),  
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])  # ImageNet Normalization
])

dataset_root = 'Datasets/Corolectal Cancer/'
if not os.path.exists(dataset_root):
    raise FileNotFoundError(f"Dataset directory '{dataset_root}' does not exist.")


full_dataset = datasets.ImageFolder(root=dataset_root, transform=transform)

# Data Profiling: Check number of classes and their album size
if len(full_dataset) == 0:
    raise ValueError("The dataset is empty. Please check the image files.")

num_classes = len(full_dataset.classes)
if num_classes == 0:
    raise ValueError("Classes Detected: FAILED")
else:
    print(f"[INFO] Classes Detected: {num_classes}, CHECK")

# Display full dataset size
class_counts = np.bincount(full_dataset.targets)
for i, count in enumerate(class_counts):
    print(f"\t> {full_dataset.classes[i]}: {count} samples")

# Data Cleansing: Conditions to meet otherwise discarded warning
min_samples = 5  
classes_to_remove = [full_dataset.classes[i] for i, count in enumerate(class_counts) if count < min_samples]
if classes_to_remove:
    print(f"[WARNING] Classes to remove due to low sample size: {classes_to_remove}")

# Data Validation: Split 80:20, DataLoaders for train/test
train_size = int(0.8 * len(full_dataset))  
test_size = len(full_dataset) - train_size  
train_dataset, test_dataset = random_split(full_dataset, [train_size, test_size])

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

def check_dataloader(dataloader, name):
    try:
        inputs, labels = next(iter(dataloader))
        print(f"[INFO] {name} DataLoader is working: CHECK\n\t> Inputs size: {inputs.size()}, \n\t> Labels size: {labels.size()}")
    except Exception as e:
        raise RuntimeError(f"[ERROR] {name} DataLoader failed: {e}")

check_dataloader(train_dataloader, "Train")
check_dataloader(test_dataloader, "Test")

[INFO] Classes Detected: 3, CHECK
	> MUS: 2000 samples
	> NORM: 2000 samples
	> STR: 2000 samples
[INFO] Train DataLoader is working: CHECK
	> Inputs size: torch.Size([32, 3, 224, 224]), 
	> Labels size: torch.Size([32])
[INFO] Test DataLoader is working: CHECK
	> Inputs size: torch.Size([32, 3, 224, 224]), 
	> Labels size: torch.Size([32])


#### SGD + Momentum Optimizations

In [16]:
# Training loop
print("[INFO] Starting training w/SGD + Momentum... CHECK")

resnet_corolectal_cancer_model.train() 

sgd_momentum_losses = []

sgd_momentum_accuracies = []

for epoch in range(num_epochs):  
    total_epoch_loss = 0.0  # Track loss for the epoch
    
    total_epoch_acc = 0.0  # Track accuracy for the epoch

    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device) 

        optimizer.zero_grad() 
        
        outputs = resnet_corolectal_cancer_model(inputs)  # Forward pass
        loss = nn.CrossEntropyLoss()(outputs, labels)  # Calculate loss
        loss.backward()  # Backward pass to compute gradients
        
        optimizer.step()  # Update model parameters using gradients

        total_epoch_loss += loss.item()

        softmax_confidence_scores = torch.nn.functional.softmax(outputs, dim=1)
        label_prediction = torch.argmax(softmax_confidence_scores, dim=1).view(-1)
        total_epoch_acc += ((label_prediction == labels).sum().item() / len(labels)) * 100

    num_batches = len(train_dataloader)
    sgd_momentum_losses.append(total_epoch_loss / num_batches)

    sgd_momentum_accuracies.append(total_epoch_acc / num_batches)

    print(f"[INFO] Epoch {epoch + 1}/{num_epochs}, Loss: {sgd_momentum_losses[epoch]:.4f}, Accuracy: {sgd_momentum_accuracies[epoch]:.4f}%")

[INFO] Starting training w/SGD + Momentum... CHECK
[INFO] Epoch 1/1, Loss: 1.1436, Accuracy: 33.4167


#### RMSProp Optimizations

In [ ]:
# Training loop
print("[INFO] Starting training... CHECK")
for epoch in range(num_epochs):  
    resnet_corolectal_cancer_model.train() 
    total_epoch_loss = 0.0  # Track loss for the epoch

    for inputs, labels in train_dataloader:
        inputs, labels = inputs.to(device), labels.to(device) 

        optimizer.zero_grad() 
        
        outputs = resnet_corolectal_cancer_model(inputs)  # Forward pass
        
        loss = nn.CrossEntropyLoss()(outputs, labels)  # Calculate loss
        
        loss.backward()  # Backward pass to compute gradients
        optimizer.step()  # Update model parameters using gradients

        total_epoch_loss += loss.item()  

    print(f"[INFO] Epoch {epoch + 1}/{num_epochs}, Loss: {total_epoch_loss / len(train_dataloader):.4f}")

# Evaluation
print("[INFO] Starting evaluation... CHECK")
resnet_corolectal_cancer_model.eval()  
with torch.no_grad(): 
    test_loss = 0.0 
    for inputs, labels in test_dataloader:
        inputs, labels = inputs.to(device), labels.to(device)  
        
        outputs = resnet_corolectal_cancer_model(inputs)  # Forward pass
        loss = nn.CrossEntropyLoss()(outputs, labels)  
        
        test_loss += loss.item()  

    average_test_loss = test_loss / len(test_dataloader)
    print(f"[INFO] Test Loss: {average_test_loss:.4f}, CHECK")